# eidein

Interactive tool for dimensionality reduction of astronomical data and their visualisation.

## TODO List

1. implement BNN and prediction uncertanties
1. subset (lasso) selection (widgets.SelectMultiple)
1. divide features (for dimensionality reduction) and spectra (for visualisation, ploting function)
1. scale to Big Data

In [1]:
%matplotlib widget

from astropy.modeling import models, fitting
import h5py
import matplotlib.pyplot as plt
import numpy as np

from eidein import Eidein
import plot

## Data

In [2]:
SIZE = 2048

In [27]:
# TODO should I take only those with high S/N?
with h5py.File("data/dr16q_superset.hdf5", "r") as datafile:
    print(datafile.keys())
    n = datafile["id"].shape[0]
    entropy = datafile["entropy"][:]
    idx = np.sort(np.argsort(entropy)[-2048:])
    entropy = entropy[idx]
    ids = datafile["id"][idx]
    X = datafile["X"][idx]
    z_pred = datafile["z_pred"][idx]
ids.dtype, X.dtype, entropy.dtype, z_vi.dtype

<KeysViewHDF5 ['X', 'entropy', 'entropy_std', 'id', 'mutual_information', 'pipe_corr_10k', 'sn_median_all', 'source_z', 'variation_ratio', 'z', 'z_10k', 'z_pca', 'z_pipe', 'z_pred', 'z_pred_std', 'z_qn', 'z_vi', 'zs_pred']>


(dtype('int32'), dtype('float32'), dtype('float64'), dtype('float32'))

In [28]:
rnd_i = np.random.randint(SIZE)
fig, ax = plt.subplots()
plot.spectrum(ax, X[rnd_i], label="Z_VI = " + str(z[rnd_i]))
plot.redshift(ax, z_pred[rnd_i])
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
Eidein(ids, X, entropy)

Eidein(children=(Tab(children=(interactive(children=(Checkbox(value=False, description='whiten'), Dropdown(des…